In [1]:
from pysal.lib import weights  # Spatial weights
from pysal.explore import esda  # Exploratory Spatial analytics
from splot import esda as esdaplot # Exploratory spatial data analysis

c:\Users\Juanes\miniforge3\envs\TemporalClusterNet\Lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
c:\Users\Juanes\miniforge3\envs\TemporalClusterNet\Lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# Ignore warnings (primarily for future deprecation warnings)
import warnings
# warnings.filterwarnings('ignore')

# Raster Analysis
import rasterio
import earthpy.plot as ep
from rasterio.merge import merge

# Vector Analysis
from shapely.geometry import box, mapping, Polygon

# General data manipulation
import geopandas as gpd
import pandas as pd
import numpy as np

from collections import Counter

# Saving and Accessing Data
import os
import pickle
import json

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import folium
import seaborn as sns

# Google Earth Engine
import ee
import geemap

# Machine learning

# Keras
import keras
from keras import Sequential
from keras.layers import Conv1D, Dropout, Dense, Input, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from importlib import reload
from tqdm import tqdm

from sklearn.decomposition import PCA

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, os.path.join('src'))

from split_images import split_geometry, haversine, calc_segment_count
import feature_extraction as fe

import sqlite3

from analysis_image import AnalysisImage
from keras.preprocessing import image

c:\Users\Juanes\miniforge3\envs\TemporalClusterNet\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [3]:
ee.Authenticate()
ee.Initialize()

In [4]:
SEED = 2024
keras.utils.set_random_seed(SEED)
FEATURES = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
years = [2018, 2019, 2020, 2021, 2022, 2023]

from tensorflow.keras.applications import ResNet152V2 as resnet
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_p

In [5]:
import eloisa
reload(eloisa)
from eloisa import Eloisa

In [6]:
# Load eloisa_databases/tegu_200x200_flatten.pkl
with open('eloisa_databases/tegu_200x200_flatten.pkl', 'rb') as f:
    tegu_200_eloisa = pickle.load(f)

tegu_200_eloisa._database = sqlite3.connect(tegu_200_eloisa.db_path)

In [8]:
# Open kmeans_50.pkl\
with open('kmeans_50.pkl', 'rb') as f:
    kmeans = pickle.load(f)

In [35]:
cluster_centroids = kmeans.cluster_centers_

# Step 2: Apply PCA to the cluster centroids
pca = PCA(n_components=1)
centroids_pca = pca.fit_transform(cluster_centroids)

sorted_indices = np.argsort(centroids_pca.flatten())

sorted_clusters = {i: float(centroids_pca[i]) for i in sorted_indices}

cluster_counts = tegu_200_eloisa.get_cluster_counts(years=years, model=resnet)

# For each key in sorted_clusters, if the key is in cluster_counts and the value is less than 1, remove the key from sorted_clusters
sorted_clusters = {k: v for k, v in sorted_clusters.items() if k in cluster_counts and cluster_counts[k] > 10}

C:\Users\Juanes\AppData\Local\Temp\ipykernel_8580\1414246370.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sorted_clusters = {i: float(centroids_pca[i]) for i in sorted_indices}


In [18]:
import cluster_tester
reload(cluster_tester)
from cluster_tester import ClusterTester

In [19]:
tegucigalpa_image = AnalysisImage(presets='Tegucigalpa', year=2023, feature_bands=FEATURES)

img_height = haversine(coords=tegucigalpa_image.get_bounds(side='ceiling'))
img_width = haversine(coords=tegucigalpa_image.get_bounds(side='left'))

x_num_parts, y_num_parts = calc_segment_count(img_height, img_width, 200, 200)

subgeometries = split_geometry(tegucigalpa_image.geometry_sd, x_num_parts=x_num_parts, y_num_parts=y_num_parts)

In [20]:
precario_path = os.path.join("data", "tegucigalpa_PC.zip")
precario_gdf = gpd.read_file(precario_path)

In [21]:
# Get cluster location multipolygons
cluster_location_info = tegu_200_eloisa.get_cluster_location_info(year=2023, model=resnet, subgeometries=subgeometries, dissolve_by_cluster=True)

In [37]:
cluster_location_info_undissolved = tegu_200_eloisa.get_cluster_location_info(year=2023, model=resnet, subgeometries=subgeometries, dissolve_by_cluster=False)

In [38]:
# For each row in cluster_location_info_undissolved, add a new column 'centroid_pca' with the value of the cluster's pca value
cluster_location_info_undissolved['centroid_pca'] = cluster_location_info_undissolved['cluster'].map(sorted_clusters)

In [44]:
# Save as geojson
cluster_location_info_undissolved.to_file("data/cluster_location_info.geojson", driver='GeoJSON')

In [43]:
cluster_location_info_undissolved

,path,image_num,cluster,geometry,centroid_pca
0,image_clips\tegucigalpa\200x200\2023\tegucigal...,0,15,"POLYGON ((-87.26837 14.03122, -87.26837 14.032...",-18.018745
1,image_clips\tegucigalpa\200x200\2023\tegucigal...,1,15,"POLYGON ((-87.26837 14.03299, -87.26837 14.034...",-18.018745
2,image_clips\tegucigalpa\200x200\2023\tegucigal...,10,15,"POLYGON ((-87.26837 14.04892, -87.26837 14.050...",-18.018745
3,image_clips\tegucigalpa\200x200\2023\tegucigal...,100,15,"POLYGON ((-87.26654 14.11793, -87.26654 14.119...",-18.018745
4,image_clips\tegucigalpa\200x200\2023\tegucigal...,1000,48,"POLYGON ((-87.23354 14.08608, -87.23354 14.087...",-19.164619
5,image_clips\tegucigalpa\200x200\2023\tegucigal...,1001,14,"POLYGON ((-87.23354 14.08784, -87.23354 14.089...",-19.310339
6,image_clips\tegucigalpa\200x200\2023\tegucigal...,1002,39,"POLYGON ((-87.23354 14.08961, -87.23354 14.091...",-18.830397
7,image_clips\tegucigalpa\200x200\2023\tegucigal...,1003,14,"POLYGON ((-87.23354 14.09138, -87.23354 14.093...",-19.310339
8,image_clips\tegucigalpa\200x200\2023\tegucigal...,1004,14,"POLYGON ((-87.23354 14.09315, -87.23354 14.094...",-19.310339
9,image_clips\tegucigalpa\200x200\2023\tegucigal...,1005,14,"POLYGON ((-87.23354 14.09492, -87.23354 14.096...",-19.310339


In [22]:
tegu_cluster_tester = ClusterTester(cluster_location_info=cluster_location_info, 
                                    analysis_image=tegucigalpa_image, 
                                    validation_data=precario_gdf, 
                                    cluster_order=sorted_clusters, 
                                    num_clusters=50,
                                    palette='jet')

In [23]:
tegu_cluster_tester.plot_clusters()